# Importation des carreaux INSEE


In [1]:
import geopandas as gpd
import pandas as pd
import requests
from io import BytesIO  # Assurez-vous d'importer BytesIO
from zipfile import ZipFile
import tempfile
import os
import matplotlib.pyplot as plt

In [2]:
url_zip = "https://drive.google.com/uc?export=download&id=1C0IweWjJaByKAswSThauqli1NbXYtzRC"
response = requests.get(url_zip)

# Créer un répertoire temporaire pour stocker les fichiers
with tempfile.TemporaryDirectory() as tmpdirname:
    with ZipFile(BytesIO(response.content)) as zipfile:
        # Extraire tous les fichiers dans le répertoire temporaire
        zipfile.extractall(tmpdirname)

        # Rechercher le fichier .shp dans le répertoire temporaire
        for file_name in os.listdir(tmpdirname):
            if file_name.endswith('.shp'):
                shp_path = os.path.join(tmpdirname, file_name)
                data = gpd.read_file(shp_path)  # Lire le fichier .shp


ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


                             Idcar_1km  I_est_1km    Ind  Men_1ind  Men_5ind  \
0      CRS3035RES1000mN2778000E3841000          1   23.0       4.8       0.0   
1      CRS3035RES1000mN2778000E3842000          1   13.0       0.9       0.5   
2      CRS3035RES1000mN2778000E3843000          0   90.0       4.9       2.0   
3      CRS3035RES1000mN2779000E3834000          0   99.0      13.0       1.0   
4      CRS3035RES1000mN2779000E3835000          0  112.0      15.0       2.0   
...                                ...        ...    ...       ...       ...   
17421  CRS3035RES1000mN2959000E3725000          0  246.5      29.0       8.0   
17422  CRS3035RES1000mN2959000E3726000          0  194.5      17.9       4.0   
17423  CRS3035RES1000mN2959000E3727000          1  201.0      18.9       5.7   
17424  CRS3035RES1000mN2959000E3728000          1   46.0       4.3       1.6   
17425  CRS3035RES1000mN2959000E3729000          1    9.0       0.8       0.3   

       Men_prop  Men_fmp    Ind_snv  Me

In [6]:
data.head()

,Idcar_1km,I_est_1km,Ind,Men_1ind,Men_5ind,Men_prop,Men_fmp,Ind_snv,Men_surf,Men_coll,...,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,Men_pauv,Men,lcog_geo,selection,geometry
0,CRS3035RES1000mN2778000E3841000,1,23.0,4.8,0.0,9.6,0.7,603810.8,1277.7,0.7,...,0.7,4.8,6.0,1.2,0.0,0.7,11.5,8928289200,1,"POLYGON ((742652.218 6758728.426, 742560.540 6..."
1,CRS3035RES1000mN2778000E3842000,1,13.0,0.9,0.5,4.1,0.0,292392.8,628.1,0.0,...,1.3,2.3,1.8,0.0,0.0,0.5,4.9,8920089282,1,"POLYGON ((743647.340 6758822.504, 743555.669 6..."
2,CRS3035RES1000mN2778000E3843000,0,90.0,4.9,2.0,28.0,0.0,2114234.0,3647.0,0.0,...,16.1,13.0,11.0,2.1,0.0,1.0,33.9,8920089268,1,"POLYGON ((744642.460 6758916.571, 744550.795 6..."
3,CRS3035RES1000mN2779000E3834000,0,99.0,13.0,1.0,36.0,4.0,2289892.1,3748.0,0.0,...,18.0,20.0,14.0,4.0,0.0,3.0,42.0,8908589152,1,"POLYGON ((735594.600 6759064.463, 735502.871 6..."
4,CRS3035RES1000mN2779000E3835000,0,112.0,15.0,2.0,39.0,4.0,2546506.8,4404.9,4.0,...,19.0,15.1,22.0,5.0,0.0,2.0,48.0,89085,1,"POLYGON ((736589.739 6759158.626, 736498.016 6..."


In [ ]:
data.plot()